This notebook is for the Capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
#Import additional packages
from sklearn.preprocessing import StandardScaler
import time
import seaborn as sns
from tqdm.auto import tqdm, trange

!pip install pyscreenshot

#Set up parallel processing
import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()
print("There are "+str(num_processes)+" logical cores on this system.")
global_starttime = time.time()

     |████████████████████████████████| 81kB 8.0MB/s eta 0:00:011
There are 56 logical cores on this system.


# Data acquisition

This project aims to provide insight in the the factors that influence the occurance and the severity of road traffic accidents specifically in the Seattle area. For this data off road traffic accidents between 2004-2020 was used. This dataset was obtained from the Seattle Open Data portal here:http://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0
A PDF with additional information is available as well, in this details of the attribute metadata can be obtained.

In [7]:
#Read data from the downloaded portal dataset and import to a dataframe
df = pd.read_csv("Collisions.csv")
#Check the first few rows of data
df_shape_orig = df.shape
print("Shape of the original data frame: "+str(df.shape[0])+' x '+str(df.shape[1]))

FileNotFoundError: [Errno 2] File b'Collisions.csv' does not exist: b'Collisions.csv'